# predict philly property price

The goal of this project is to predict the value of homes in Philadelphia and identify features that drive up home value. 

Property data is sourced from OpenDataPhilly and the City of Philadelphia Office of Property Assement: https://www.phila.gov/property/data/#

Info features in the dataset were downloaded from this url: https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/

In addition to the information in the above dataset, additional information on school locations will be used to calculate proximity to schools: https://www.opendataphilly.org/dataset/schools/resource/8e1bb3e6-7fb5-4018-95f8-63b3fc420557

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import datetime as dt
import time
from sklearn.neighbors import BallTree
from sklearn.metrics import DistanceMetric 
from geopy import distance as geo_distance
import pickle

In [2]:
df = pd.read_csv('raw_data/opa_properties_public.csv',low_memory=False)
var = pd.read_csv('raw_data/fields.csv',low_memory=False)
var = var.set_index('Field Name',drop=True)
df = df.rename(columns={'view_type':'view'})

Drop unnecessary data columns:

In [3]:
droplist = ['the_geom','the_geom_webmercator','book_and_page','building_code','building_code_description',\
            'census_tract','cross_reference',\
            'date_exterior_condition','homestead_exemption',\
            'house_number','parcel_number','parcel_shape',\
            'registry_number','suffix','state_code','street_code',\
            'topography','pin','objectid','unit','owner_1','owner_2','zoning',\
            'beginning_point','exempt_land','exempt_building',\
            'general_construction','mailing_address_1',\
            'mailing_address_2','mailing_care_of','mailing_city_state',\
            'mailing_street','recording_date','sale_date','sale_price',\
            'site_type','street_name','taxable_building','other_building',\
            'house_extension','separate_utilities','taxable_land','market_value_date',\
            'off_street_open','location','market_value_date','fireplaces','geographic_ward']
df = df.drop(droplist,axis=1)

In [4]:
for row in var.index:
    if row not in df.columns:
        var = var.drop(row)

Wrote a dictionary of dictionaries based on more length text info in fields.csv

In [5]:
catdict = {'basements':{'0':'none','A':'full finished','B':'full semi-finished','C':'full unfinished','D':'full unknown finish',\
                       'E':'partial finished','F':'partial semi-finished','G':'partial unfinished','H':'partial unknown finish',\
                       'I':'finished-unknown size','J':'unfinished-unknown size'},\
          'garage_type':{'0':'none','A':'built in basement','B':'attached garage','C':'detached garage',\
                        'F':'converted'},\
          'type_heater':{'H':'undetermined','A':'hot air (ducts)','B':'hot water (radiators/baseboards)','G':'radiant',\
                         'C':'electric baseboard','E':'other','D':'heat pump (outside unit)'},\
          'view':{'0':'na','I':'typical','A':'cityscape','C':'park/green area','B':'flowing water'}}

Filtering data to only consider single family homes (exclude vacant land, multiuse props etc):

In [6]:
df = df[df['category_code_description'].isin(['Single Family'])]
df  = df.drop(['category_code_description','category_code'],axis=1)

Drop rows where the property is unfinished:

In [7]:
df = df[df.unfinished != 'U']

Drop rows where there is no estimate of the year the property was built:

In [8]:
df = df[df['year_built_estimate'] == 'Y']
df = df.drop('year_built_estimate',axis=1)
df['year_built'] = df['year_built'].astype('int')

Drop columns where more than half of the rows are NaNs:

In [9]:
nas_dropcol = list()
for column in df:
    if df[column].isna().sum()/len(df[column]) > 0.5:
        nas_dropcol.append(column)
df = df.drop(nas_dropcol,axis=1)
for row in var.index:
    if row not in df.columns:
        var = var.drop(row)

In [10]:
print('will use saved csv file. if columns change, need to update the datatype in the csv file and reload')
#time = dt.datetime.now()
#ts = "{:d}{:02d}".format(time.hour, time.minute)
#fname = 'interim_data/fields_stripped_' + ts + '.csv'
#var.to_csv(fname)

will use saved csv file. if columns change, need to update the datatype in the csv file and reload


In [12]:
print('loading data types...')
ts = '1552'
fname = 'interim_data/fields_stripped_' + ts + '.csv'
ivar = pd.read_csv(fname)
ivar = ivar.set_index('Field Name')

loading data types...


Fix data type using 'Type' column from ivar df (from csv file):

In [13]:
for column in df:
    mantype = ivar['Type'][ivar.index == column][0]    
    if mantype == 'cat':
        df[column] = df[column].astype('category',errors='ignore')
    elif mantype == 'num':
        df[column] = df[column].astype('float')
    elif mantype == 'logical':
        df[column] = df[column].replace({'Y':1,'N':0,'y':1,'n':0,'1':1,'0':0})
        df[column] = df[column].astype('category',errors='ignore')
    elif mantype == 'date':
        continue
    else:
        print(column + " needs data type")

Check for columns that still have a large proportion of missing values:

In [14]:
prop_na = df.isna().sum().sort_values(ascending=False)/df.shape[0]
pndf = pd.DataFrame(data=prop_na,columns=['propna'])
pndf = pndf.reset_index()
pndf.head()

,index,propna
0,central_air,0.447066
1,type_heater,0.412132
2,basements,0.339402
3,number_of_rooms,0.101401
4,garage_type,0.100679


Fill the the below columns with their mode (excluding NaNs):

In [15]:
df['central_air'] = df['central_air'].fillna(int(df['central_air'].mode(dropna=True)))
heat_mode = df['type_heater'].mode(dropna=True).values[0]
df['type_heater'] = df['type_heater'].fillna(value=heat_mode)
base_mode = df['basements'].mode(dropna=True).values[0]
df['basements'] = df['basements'].fillna(value=base_mode)
gar_mode = df['garage_type'].mode(dropna=True).values[0]
df['garage_type'] = df['garage_type'].fillna(value=gar_mode)

If the number of bathrooms or rooms is 0, assume it's important missing info and drop

In [16]:
df = df[df['number_of_rooms'] > 0]
df = df[df['number_of_bathrooms'] > 0]
df = df[df['number_stories'] > 0]

Now how many nans remain across the columns?

In [17]:
prop_na = df.isna().sum().sort_values(ascending=False)/df.shape[0]
pndf = pd.DataFrame(data=prop_na,columns=['propna'])
pndf = pndf.reset_index()
pndf.head()

,index,propna
0,view,0.000044
1,garage_spaces,0.000013
2,interior_condition,0.000013
3,basements,0.000000
4,number_stories,0.000000


Because the proportions of nans are minimal for those that remain, drop these rows from the dataset:

In [19]:
df = df.dropna(axis=0)

Confirm the data is free of nans now:

In [20]:
if df.isna().sum().any():
    print('fix null values')
else:
    print('free of null values')
print('new shape: ' + str(df.shape))

free of null values
new shape: (228899, 22)


Get rid of outliers:

In [21]:
def getBoundsForOutlierDetection(df,column,qlo,qhi):
    lobound = df[column].quantile(qlo)
    hibound = df[column].quantile(qhi)
    iqr = hibound - lobound
    upper = hibound + 1.5 * iqr
    lower = lobound - 1.5 * iqr
    return lower, upper

In [22]:
df['zip_code'] = df['zip_code'].astype('int')
ignore = ['lat','lng']

In [23]:
graded  = ['interior_condition','exterior_condition']
schdist = ['km_to_elementary', 'km_to_middle', 'km_to_high']
loc     = ['lat','lng']
exclude = list()
for i in schdist:
    exclude.append(i)
for i in loc:
    exclude.append(i)
for i in graded:
    exclude.append(i)   
predictors = [column for column in df.select_dtypes('float').columns if column not in exclude]

In [24]:
discrete  = list()
continous = list()
for x in predictors:
    if x.__contains__('number'):
        discrete.append(x)
discrete.append('garage_spaces')

for x in predictors:
    if x not in discrete:
        continous.append(x)
continous.append('year_built')

In [25]:
for column in continous:
    if column in ignore:
        continue
    lower, upper = getBoundsForOutlierDetection(df,column,qlo=0.25,qhi=0.75)
    df = df[(df[column] >= lower) & (df[column] <= upper)]

Drop values not in dictionary for columns in dictionary (missing info):

In [26]:
these_columns = list(catdict.keys())
for column in these_columns:
    unqvals = list(df[column].unique())
    ok_vals = list(catdict[column].keys())
    rm_vals = [x for x in unqvals if x not in ok_vals]
    for value in rm_vals:
        df = df[df[column] != value]
    print(column + ' values remaining after drop: ' + str(df.shape[0]))

basements values remaining after drop: 173001
garage_type values remaining after drop: 172985
type_heater values remaining after drop: 172268
view values remaining after drop: 169548


Check value counts for categorical variables and drop rarely occuring values:

In [27]:
for catcol in df.select_dtypes('category').columns:
    prop = df[catcol].value_counts(normalize=True).reset_index()
    rm_vals = list(prop['index'][prop[catcol] < 0.005])
    for value in rm_vals:
        df = df[df[catcol] != value]

In [28]:
for column in discrete:
    prop = df[column].value_counts(normalize=True).reset_index()
    rm_vals = list(prop['index'][prop[column] < 0.005])
    for value in rm_vals:
        df = df[df[column] != value]

Now subsample the dataframe for easier computing (running locally):

In [29]:
df = df.sample(n=15000)

Read in location info for schools:

In [30]:
schools = pd.read_csv('raw_data/Schools.csv',low_memory=False)
schools=schools.drop(['OBJECTID','AUN','SCHOOL_NUM','SCHOOL_NAME',\
                     'SCHOOL_NAME_LABEL','STREET_ADDRESS',\
                     'ZIP_CODE','PHONE_NUMBER','GRADE_ORG',\
                     'ENROLLMENT','TYPE','LOCATION_ID','ACTIVE'],axis=1)
schools.rename(columns={'X':'lng','Y':'lat'},inplace=True)
schools['elementary'] = schools['GRADE_LEVEL'].str.contains('ELEMENTARY')
schools['middle'] = schools['GRADE_LEVEL'].str.contains('MIDDLE')
schools['high'] = schools['GRADE_LEVEL'].str.contains('HIGH')

In [31]:
def findShortestDistToSchool(schools,df,school_type):
    start = time.time()
    filt_schools = schools[['lat','lng']][schools[school_type] == True]
    coords = np.radians(filt_schools[['lat', 'lng']])
    tree = BallTree(coords, metric=DistanceMetric.get_metric('cityblock'))
    minll = list()
    minkm = list()
    nearestschool = list()
    for idx in df.index:
        house_coord = np.array(df[['lat','lng']].loc[idx])
        r_house_coord = np.radians(house_coord)
        mindist, near_school_idx = tree.query([r_house_coord], k=1)
        nsi = int(near_school_idx)
        nearestschool.append(np.array(filt_schools[['lat','lng']].iloc[nsi]))
        minll.append(mindist)
        minkm.append(geo_distance.distance(np.array(filt_schools[['lat','lng']].iloc[nsi]),house_coord).km)
    print('elapsed t: ' + str(time.time() - start))
    return minll,minkm,nearestschool

In [32]:
ell,ekm,ens = findShortestDistToSchool(schools,df,'elementary')
mll,mkm,mns = findShortestDistToSchool(schools,df,'middle')
hll,hkm,hns = findShortestDistToSchool(schools,df,'high')

elapsed t: 15.296080112457275
elapsed t: 15.664908647537231
elapsed t: 19.819714784622192


In [33]:
df['km_to_elementary'] = ekm
df['km_to_middle'] = mkm
df['km_to_high'] = hkm

In [34]:
df.to_pickle('interim_data/df_cleaned')